## Membangun Chatbot
Dalam Materi ini, kita akan membahas contoh cara mendesain dan mengimplementasikan chatbot berbasis LLM. Chatbot ini akan mampu melakukan percakapan dan mengingat interaksi sebelumnya.

Perhatikan bahwa chatbot yang kita bangun ini hanya akan menggunakan model bahasa untuk melakukan percakapan. Ada beberapa konsep terkait lainnya yang mungkin Anda cari:

- RAG Percakapan: Mengaktifkan pengalaman chatbot melalui sumber data eksternal
- Agen: Membangun chatbot yang dapat mengambil tindakan

Tutorial Materi ini akan membahas dasar-dasar yang akan bermanfaat untuk dua topik yang lebih lanjut tersebut.

In [23]:
import os
from dotenv  import load_dotenv

load_dotenv()##load environment variables from a .env file

openai_api_key = os.getenv("OPENAI_API_KEY")

In [24]:
from langchain_openai import ChatOpenAI
model=ChatOpenAI(model="gpt-5-nano-2025-08-07",openai_api_key=openai_api_key)
model

ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x00000288FFC61F90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000288FFC63310>, root_client=<openai.OpenAI object at 0x00000288FFC629B0>, root_async_client=<openai.AsyncOpenAI object at 0x00000288FFC615D0>, model_name='gpt-5-nano-2025-08-07', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [25]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Halo, nama saya Novandra dan saya merupakan pelajar AI.")])

AIMessage(content='Halo Novandra! Senang bertemu dengan seorang pelajar AI. Ada yang bisa saya bantu hari ini?\n\nSaya bisa bantu dengan:\n- Penjelasan konsep AI/ML secara singkat maupun mendalam\n- Rencana belajar khusus (misalnya 6–8 minggu untuk pemula)\n- Bantuan tugas/proyek AI\n- Saran sumber belajar (buku, kursus, dokumentasi)\n- Contoh kode latihan (Python, ML dengan scikit-learn, PyTorch/TensorFlow)\n\nBeberapa pertanyaan untuk menyesuaikan bantuan:\n- Tujuan utama Anda (persiapan kuliah, tugas, atau projek pribadi)?\n- Level saat ini (pemula, menengah)?\n- Bahasa pemrograman yang Anda pakai (umumnya Python)?\n- Topik yang ingin difokuskan (misalnya ML dasar, deep learning, computer vision, NLP, dsb)?\n- Apakah Anda ingin rencana belajar singkat (mis. 4–6 minggu) atau proyek/latihan praktik?\n\nKalau Anda mau, saya bisa langsung buatkan contoh rencana belajar 6–8 minggu untuk pemula dengan Python, ML dasar (supervised learning), dan proyek mini. Ingin saya buatkan sekarang?', 

In [26]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Halo, nama saya Novandra dan saya merupakan pelajar AI."),
    AIMessage(content="Halo Novandra! Senang bertemu denganmu. Senang mendengar kamu mahasiswa AI.\n\nMau saya bantu dengan apa hari ini? Beberapa opsi yang bisa saya tawarkan:\n- Penjelasan konsep AI/ML (mis. supervised vs unsupervised, neural networks, backpropagation)\n- Bantu memahami materi kuliah atau tugas tugas kuliah\n- Ide proyek AI untuk mahasiswa (contoh: sentiment analysis, image classifier sederhana, atau recommender system)\n- Contoh kode Python untuk tugas/latihan (menggunakan NumPy, PyTorch, atau TensorFlow)\n- Rencana belajar singkat atau sumber belajar yang relevan\n\nKalau ada topik spesifik yang kamu minati sekarang, kasih tahu ya. Atau jika mau, saya bisa mulai dengan penjelasan singkat tentang konsep dasar AI. Mau saya jelaskan?"),
    HumanMessage(content="siapa kah aku?")
])

AIMessage(content='Kamu Novandra, seorang pelajar AI. Senang bertemu! Mau saya bantu dengan apa hari ini? Beberapa opsi:\n- Penjelasan konsep AI/ML (misalnya supervised vs unsupervised, neural networks)\n- Bantu dengan materi kuliah atau tugas kuliah\n- Ide proyek AI untuk mahasiswa\n- Contoh kode Python untuk tugas latihan (NumPy, PyTorch, TensorFlow)\n- Menyusun rencana belajar singkat\n\nKalau ada topik spesifik yang lagi kamu pelajari, kasih tahu ya. Mau saya jelaskan konsep dasar AI dulu?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 641, 'prompt_tokens': 207, 'total_tokens': 848, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D4JhP7T3RWsRdFvdrTvQlhRzWSPeG', 'servic

### Message History
Kita dapat menggunakan kelas Message History untuk membungkus model kita dan membuatnya memiliki status (stateful). Ini akan melacak input dan output model, dan menyimpannya di suatu penyimpanan data. Interaksi selanjutnya kemudian akan memuat pesan-pesan tersebut dan meneruskannya ke dalam rantai sebagai bagian dari input. Mari kita lihat bagaimana cara menggunakannya!

In [27]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

histories = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in histories:
        histories[session_id] = ChatMessageHistory()
    return histories[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [28]:
config={"configurable":{"session_id":"chat_session_1"}}

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Halo, nama saya Novandra dan saya merupakan pelajar AI.")],
     config=config
     )

In [30]:
response.content

'Halo, Novandra! Senang bertemu dengan sesama pelajar AI. Ada yang bisa saya bantu hari ini? Apakah kamu ingin penjelasan konsep, saran rencana belajar, atau latihan praktis?\n\nBeberapa opsi yang bisa saya bantu:\n- Penjelasan konsep dasar AI/ML/DL dan perbedaannya (AI, ML, DL; supervised vs unsupervised; neural networks).\n- Pengenalan matematika penting (vektor/matriks, kalkulus dasar, probabilitas, gradient descent).\n- Panduan cepat untuk Python, NumPy/pandas, dan dasar debugging ML.\n- Perbandingan framework PyTorch vs TensorFlow dan contoh proyek sederhana.\n- Ide proyek praktis (misalnya klasifikasi gambar sederhana, regresi, NLP basic) beserta langkah implementasinya.\n- Tips evaluasi model, regularisasi, menghindari overfitting, dan etika AI.\n\nKalau kamu mau, aku bisa buatkan rencana belajar 6 minggu yang disesuaikan dengan level kamu. Beberapa pertanyaan singkat untuk menyesuaikan:\n- Kamu lebih tertarik ke ML umum, computer vision, atau natural language processing?\n- Kam

In [31]:
response=with_message_history.invoke(
    [HumanMessage(content="siapa kah aku?")],
     config=config
     )

In [32]:
response.content

'Kamu adalah Novandra, seorang pelajar AI. Senang bertemu!\n\nMau aku lanjut dengan:\n- penjelasan singkat perbedaan AI/ML/DL, atau\n- rencana belajar 6 minggu yang disesuaikan denganmu?\n\nKalau mau, kasih tahu juga:\n- fokus minat (ML umum, computer vision, NLP)\n- pengalaman pemrograman/matematika (Python, algebra, probabilitas)\n- target belajarmu (hobi, proyek kecil, persiapan kuliah/karier)'

In [33]:
### Mengganti session id untuk percakapan baru
config={"configurable":{"session_id":"chat_session_2"}}
response=with_message_history.invoke(
    [HumanMessage(content="siapa kah aku?")],
     config=config
     )
response.content

'Aku nggak bisa tahu persis siapa kamu hanya dari percakapan ini. Tapi aku bisa bantu kalau kamu ingin membuat deskripsi diri atau memahami identitasmu sendiri.\n\nKalau kamu mau, kamu bisa kasih sedikit info tentang dirimu, atau kita pakai latihan singkat ini:\n- 3 kata yang paling menggambarkan dirimu\n- Hal yang paling kamu hargai/percayai\n- Tujuan utama yang sedang kamu kejar sekarang\n- 1 pencapaian yang bikin bangga\n- 1 hal yang ingin kamu capai dalam 6–12 bulan ke depan\n\nBila kamu kasih jawabannya, aku bisa bantu merangkum jadi bio diri yang singkat, misalnya:\n"Seorang [pekerjaan/peran] yang suka [minat], percaya pada [nilai], sedang berusaha [tujuan]."\n\nMau kita mulai dengan satu atau dua pertanyaan di atas? Atau jelaskan konteksnya (misalnya buat bio LinkedIn, bio media sosial, atau refleksi pribadi) supaya aku bisa sesuaikan.'